In [21]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [12]:
load_dotenv(override=True)

api_key= os.getenv('OPENAI_API_KEY')
base_url=os.getenv('OPENAI_BASE_URL')


if api_key and api_key.startswith('helix-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL="gpt-4o"

openai = OpenAI(
    base_url=base_url,
    api_key=api_key
)

API key looks good so far


In [33]:
system_message = "You are a helpful assistant that responds in markdown"

In [34]:
def message_gpt(prompt):
    messages = [
        {"role":"system", "content":system_message},
        {"role":"user", "content":prompt}
    ]
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
message_gpt("How many pages are there in the book Macbeth")

## User Interface

## Forcing Mode

In [ ]:
force_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""


view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:")],
    js=force_mode,
    flagging_mode="never")

view.launch()

In [40]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()